# Linear classification

The files $trainA$, $trainB$ and $trainC$ contain samples of data $(x_n,y_n)$ where $x_n∈$ $\mathbb{R^2}$ and $y_n∈\{0,1\}$ (each line of each file contains the 2 components of $x_n$ then $y_n$). The goal of this exercise is to implement linear classification methods and to test them on the three data sets.

#### Util functions and libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score

In [0]:
def sigm(x):
    """ 
    This function compute the sigmoid function. 
    """
    return 1/(1+np.exp(-x))
def plot_classification(train,train,boundary_x,boundary_y,i, linear = True,
                        model,Z=0,Zt=0):
    
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    x_test = test[:,:-1]
    y_test = test[:,2]

    X_1 = x_train[y_train == 1]
    X_0 = x_train[y_train == 0]
    Xt_1 = x_test[y_test == 1]
    Xt_0 = x_test[y_test == 0]
    fig, ax =plt.subplots(1,2)
    #fig.suptitle(model +' on data ' + i )
    ax[0].set_title(model + regression train set " + i)
    if linear:
      ax[0].plot(boundary_x, boundary_y, c='g', label='Decision boundary')
    else:
      ax[0].contour(boundary_x0, boundary_x1,Z,[0] ,colors="g")
    ax[0].scatter(X_1[:, 0], X_1[:, 1], c='r', label='True label: 1')
    ax[0].scatter(X_0[:, 0], X_0[:, 1], c='b', label='True label: 0')

    ax[1].set_title(model+ " test set " + i)
    if linear:
      ax[0].plot(boundary_x, boundary_y, c='g', label='Decision boundary')
    else:
      ax[1].contour(boundary_xt0, boundary_xt1,Zt,[0], colors = "g")
    ax[1].scatter(Xt_1[:, 0], Xt_1[:, 1], c='r', label='True label: 1')
    ax[1].scatter(Xt_0[:, 0], Xt_0[:, 1], c='b', label='True label: 0')

In the following code cell, we will load the three Datasets

In [0]:
#Loading the datasets

#Training sets
trainA = np.array(pd.read_csv('trainA.csv', delimiter = " "))
trainB = np.array(pd.read_csv('trainB.csv', delimiter = " "))
trainC = np.array(pd.read_csv('trainC.csv', delimiter = " "))

list_train = [trainA, trainB, trainC]

#testing sets
testA = np.array(pd.read_csv('testA.csv', delimiter = " "))
testB = np.array(pd.read_csv('testB.csv', delimiter = " "))
testC = np.array(pd.read_csv('testC.csv', delimiter = " "))

list_test = [testA, testB, testC]


### Generative Model (LDA)


Given the class variable, the data are assumed to be Gaussian with different means for different classes but with the same covariance matrix.
$$\begin{aligned}
  y∼Bernoulli(π) , x|y=i∼Normal(μ_i,Σ).
\end{aligned}$$

\\

####  Implementation of the MLE for this model and representation of the data and the classification line defined by the equation $p(y=1|x)=0.5$ : 

In [0]:
def train_LDA(train,test):
    """
    This function train the LDA model.
    INPUT :  
        data : csv file containing the points in R² and their label y.

    OUTPUT : w : estimated weights 
             b : estimated bias

    """
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    x_test = test[:,:-1]
    y_test = test[:,2]

    n1 = sum(y_train)
    N = len(y_train)
    pi = n1/N
    mu_1 = np.sum(x_train[y_train == 1], axis=0)/n1
    mu_0 = np.sum(x_train[y_train == 0], axis=0)/(N-n1)
    sigma =1/N * ( np.dot((X_1-mu_1).T,(X_1-mu_1)) + 
                  np.dot((X_0-mu_0).T,(X_0-mu_0)))
    
    sigma_inv = np.linalg.inv(sigma)

    beta = sigma_inv @(mu_1-mu_0)
    alpha = -1/2 * (mu_1-mu_0).T @ sigma_inv @ (mu_1 + mu_0) +np.log(pi/(1-pi))

    print("pi:", round(pi,2), "\nmu_1:", mu_1, "\nmu_0:", mu_0, "\nSigma:", 
          sigma, "\nbeta:", beta, "\nalpha:", round(alpha,2))

    return beta, alpha 


 def separation_line(train,test,alpha,beta):  

    x_train = train[:,:2]
    y_train = train[:,2]
    
    x_test = test[:,:-1]
    y_test = test[:,2]
    pi = sum(y_train)/len(y_train)
    x_min, x_max = np.min(x_train[:,0]), np.max(x_train[:,0])
     
    boundary_x = np.linspace(x_min, x_max, 1000)
    boundary_y = [(np.log((1-pi)/pi)- beta[0]*x-alpha)/beta[1] 
                  for x in boundary_x]
    
    Xt_1 = x_test[y_test == 1]
    Xt_0 = x_test[y_test == 0]
    
    
    xt_min, xt_max = np.min(x_test[:,0]), np.max(x_test[:,0])
    boundary_xt = np.linspace(xt_min, xt_max, 1000)
    boundary_yt = [(np.log((1-pi)/pi)- beta[0]*x-alpha)/beta[1] 
                  for x in boundary_x]

    return boundary_x, boundary_y, boundary_xt, boundary_yt

  def predict(train,test, alpha, beta):
    pi = sum(y_train)/len(y_train)
    pred = [(np.log(pi/(1-pi)) - alpha - beta[0]*x[0])/beta[1] 
          for x in test[:,:2]]
    predict = test[:,1] > pred
    misclass = 1-sum((predict != test[:,2]))/len(predict)

    pred = [(np.log(pi/(1-pi)) - alpha - beta[0]*x[0])/beta[1] 
            for x in train[:,:2]]
    predict = train[:,1] > pred
    misclass_train = 1-sum((predict != train[:,2]))/len(predict)
    
    print("Misclassification on train set: ", misclass_train)
    print("\n")
    print("Misclassification on test set: ", misclass)
    print("\n")

In [0]:
model = "LDA"
train, test = trainA, testA
beta, alpha = train_LDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = separation_line(train,test,
                                                                   alpha,beta)
plot_classification(train,test,boundary_x,boundary_y,"A", linear = True,
                        model,Z=0,Zt=0)
predict(train,test, alpha, beta)

In [0]:
train, test = trainB, testB
beta, alpha = train_LDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = separation_line(train,test,
                                                                   alpha,beta)
plot_classification(train,test,boundary_x,boundary_y,"B", linear = True,
                        model,Z=0,Zt=0)
predict(train,test, alpha, beta)

In [0]:
train, test = trainB, testB
beta, alpha = train_LDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = separation_line(train,test,
                                                                   alpha,beta)
plot_classification(train,test,boundary_x,boundary_y,"C", linear = True,
                        model,Z=0,Zt=0)

predict(train,test, alpha, beta)

###  Logistic regression

The objective of this subsection is to implement logistic regression for an affine function $f(x)=\mathbf{w}^\intercal \mathbf{x}+\mathbf{b}$.

####  Implementation of the MLE for this model and representation of the data and the classification line defined by the equation $p(y=1|x)=0.5$.



Logistic regression works as follows.

Given an iid sample $\{(\boldsymbol{x}^{(1)}, y^{(1)}), ..., (\boldsymbol{x}^{(n)}, y^{(n)})\}$ of random variables $x$  and $y$
with 
* $\boldsymbol{x}$ a $d-$dimensional vector $\boldsymbol{x} = (x_1, ..., x_d)$. In the present case $d=2$.
* $y|X=x$ follows a Bernoulli law with parameter $\sigma (f(x)) = \sigma (\mathbf{w}^\intercal \mathbf{x}+\mathbf{b}) $, $y \in \{0,1\}$

The log-likelihood of the sample is written :
$ l(w,b) =$  $  \sum_{i=1}^{n} \{ y_i.log(\sigma (f(x_i))  + (1-y_i).log(1 -\sigma (f(x_i)) \}$. The goal is to minimize this negative log-likelihood which means to find $\hat{w}, \hat{b}  = \underset{w, b}{\operatorname{argmax}} l(w,b)$.

To find the weights and the bias, we will implement a first order gradient ascent.

In [0]:
def gradient_ascent(n_iter, data, lrate):
    """
    This function is a first order gradient ascent algorithm which estimates 
    the weights and the bias of this model.
    INPUT :  
        data : ndarray conatining points in R² and their label y.
        n_iters = number of iterations through which the update of parameters 
        using gradient descent will be done
        learning_rate : the step size used for the gradient ascent algorithm
    OUTPUT : w : estimated weights 
             b : estimated bias

    """
    x = data[:,:2]
    y= data[:,2].reshape(-1,1)
    #N = len(data)
    w = np.zeros(shape=(np.shape(x)[1],1))
    b = 0
    for i in range(n_iter):
        y_pred = sigm(np.dot(x,w) + b)
        grad_w = np.dot(x.T, y-y_pred)
        grad_b = sum(y-y_pred)
        w = w + lrate*grad_w
        b = b + lrate*grad_b
    return w, b

In [0]:
def sep_logreg(train, test):            
    w,b = gradient_ascent(100, train, 0.01)
    
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    x_test = test[:,:-1]
    y_test = test[:,2]
    
    X_1 = x_train[y_train == 1]
    X_0 = x_train[y_train == 0]
    
    x_min, x_max = np.min(x_train[:,0]), np.max(x_train[:,0])
         
    boundary_x = np.linspace(x_min, x_max, 1000)
    boundary_y = [(b- w[0]*x)/w[1] for x in boundary_x]

    Xt_1 = x_test[y_test == 1]
    Xt_0 = x_test[y_test == 0]
    
    
    xt_min, xt_max = np.min(x_test[:,0]), np.max(x_test[:,0])
    boundary_xt = np.linspace(xt_min, xt_max, 1000)
    boundary_yt = [(b- w[0]*x)/w[1] for x in boundary_x]

    return boundary_x, boundary_y, boundary_xt, boundary_yt



def pred_logreg(train, test):
  pred = [(b- w[0]*x[0])/w[1] for x in te st[:,:2]]
  predict = test[:,1].reshape(-1,1) > pred
  misclass = 1-sum((predict != test[:,2].reshape(-1,1)))/len(predict)

  pred = [(b- w[0]*x[0])/w[1] for x in train[:,:2]]
  predict = train[:,1].reshape(-1,1) > pred
  misclass_train = 1-sum((predict != train[:,2].reshape(-1,1)))/len(predict)
  print("Misclassification on train set: ", misclass_train)
  print("\n")
  print("Misclassification on test set: ", misclass)
  print("\n")
    

Let's estimate parameters for each training set using the train function, and then let's determine the line defined by P(Y=1|x) = 0.5 for each training and testing sets. Then let's plot the datasets with the separating line.  

In [0]:
model = "Logistic regression"
train, test = trainA, testA
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_logreg(train,test)
plot_classification(train,test,boundary_x,boundary_y,"A", linear = True,
                        model,Z=0,Zt=0)
pred_logreg(train,test, alpha, beta)

In [0]:
train, test = trainA, testA
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_logreg(train,test)
plot_classification(train,test,boundary_x,boundary_y,"B", linear = True,
                        model,Z=0,Zt=0)
pred_logreg(train,test, alpha, beta)

In [0]:
train, test = trainA, testA
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_logreg(train,test)
plot_classification(train,test,boundary_x,boundary_y,"C", linear = True,
                        model,Z=0,Zt=0)
pred_logreg(train,test, alpha, beta)

###  Linear regression

The objective of this subsection is to implement linear regression for an affine function $f(x)=\mathbf{w}^\intercal \mathbf{x}+\mathbf{b}$ by solving normal equations.

Linear regression works as follows.

Given an iid sample $\{(\boldsymbol{x}^{(1)}, y^{(1)}), ..., (\boldsymbol{x}^{(n)}, y^{(n)})\}$ of random variables $x$  and $y$
with 
* $\boldsymbol{x}$ is a $d-$dimensional vector $\boldsymbol{x} = (x_1, ..., x_d)$. In the present case $d=2$.
* $y_i = x_i^\intercal .w+ b  = \left(
\begin{array}{c}
x_i\\
1\\
\end{array}
\right) ^\intercal. \ \left(
\begin{array}{c}
w\\
b\\
\end{array}
\right) .$

This can be written in matricial terms : $ y= X.\beta $ where the $i^{th}$ row of matrix $X$ is $ \left( \begin{array}{r} x_i\\ 1\\ \end{array} \right) ^T$; and $\beta  = \left(
\begin{array}{c}
w\\
b\\
\end{array}
\right)$. 

 The objective is to find $\hat{\beta}  = \underset{\beta}{\operatorname{argmin}} \frac{1}{2n} \left\Vert y- X.\beta \right\Vert^2 $. To solve this equation, we will solve normal equation : $X ^\intercal X \beta = X ^\intercal y \implies \beta = (X^\intercal.X)^{-1}.X^\intercal.y $ . 

In [0]:
def train_linreg(train,test)            
            
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    x_test = test[:,:-1]
    y_test = test[:,2]
    
    X = np.append(x_train,np.ones(len(x_train)).reshape(-1,1),axis=1)
    theta = np.linalg.inv(X.T @ X)@X.T @ y_train.reshape(-1,1)
    w, b = theta[:2,],theta[2,]
    return w, b 
    
def sep_linreg(train,test, w, b )

  x_train = train[:,:-1]
  y_train = train[:,2]
  
  x_test = test[:,:-1]
  y_test = test[:,2]
  X_1 = x_train[y_train == 1]
  X_0 = x_train[y_train == 0]

  x_min, x_max = np.min(x_train[:,0]), np.max(x_train[:,0])
      
  boundary_x = np.linspace(x_min, x_max, 1000)
  boundary_y = [(1/2 -b- w[0]*x)/w[1] for x in boundary_x]

  xt_min, xt_max = np.min(x_test[:,0]), np.max(x_test[:,0])
      
  boundary_x = np.linspace(xt_min, xt_max, 1000)
  boundary_y = [(1/2 -b- w[0]*x)/w[1] for x in boundary_x]


def pred_linreg(train,test, w, b):

  #compute misclassification for train_set
  pred = [(1/2 -b- w[0]*x)/w[1] for x in train[:,0]]
  predict = train[:,1].reshape(-1,1) > pred
  misclass_train = 1-sum((predict != train[:,2].reshape(-1,1)))/len(predict)

  #compute misclassification for test_set
  pred = [(1/2 -b- w[0]*x)/w[1] for x in test[:,0]]
  predict = test[:,1].reshape(-1,1) > pred
  misclass = 1-sum((predict != test[:,2].reshape(-1,1)))/len(predict)
  
  print("w = ", w)
  print("b = ", b)
  
  print("Misclassification on train set: ", misclass_train)
  print("\n")
  print("Misclassification on test set: ", misclass)
  print("\n")


Let's estimate parameters for each training set using the train function, and then let's determine the line defined by P(Y=1|x) = 0.5 for each training and testing sets. Then let's plot the datasets with the separating line. 

In [0]:
model = "Linear regression"
train, test = trainA, testA
w, b = train_linreg(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_linreg(train,test,
                                                                   w,b)
plot_classification(train,test,boundary_x,boundary_y,"A", linear = True,
                        model,Z=0,Zt=0)
pred_linreg(train,test, w, b)

In [0]:
model = "Linear regression"
train, test = trainA, testA
w, b = train_linreg(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_linreg(train,test,
                                                                   w,b)
plot_classification(train,test,boundary_x,boundary_y,"B", linear = True,
                        model,Z=0,Zt=0)
pred_linreg(train,test, w, b)

In [0]:
model = "Linear regression"
train, test = trainA, testA
w, b = train_linreg(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt = sep_linreg(train,test,
                                                                   w,b)
plot_classification(train,test,boundary_x,boundary_y,"C", linear = True,
                        model,Z=0,Zt=0)
pred_linreg(train,test, w, b)

### QDA model

We finally relax the assumption that the covariance matrices for the two classes are the same. So, given the class label the data are assumed to be Gaussian with means and covariance matrices which are a priori different.
$ y \sim \mathcal{B} {(\pi)}$ , $x|y=i$ $\sim \mathcal{N}(\mu_i ,\,\Sigma _i)\,$

The goal of this subsection is to implement the maximum likelihood estimator of this model and apply it to the data. 


####  Implementation of the MLE for this model and representation of the data and the classification line defined by the equation $p(y=1|x)=0.5$ :

In [0]:
def train_QDA(train,test):
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    y = y_train.reshape(-1,1)
    x_test = test[:,:-1]
    y_test = test[:,2]
    
    X_1 = x_train[y_train == 1]
    X_0 = x_train[y_train == 0]

    n1 = sum(y_train)
    N = len(y_train)
    pi = n1/N
    mu_1 = np.sum(X_1, axis=0)/n1
    mu_0 = np.sum(X_0, axis=0)/(N-n1)
    sigma_1 =1/n1 * np.dot((X_1-mu_1).T,(X_1-mu_1))
    sigma_0 = 1/(N-n1) * np.dot((X_0-mu_0).T,(X_0-mu_0))
    
    sigma0_inv = np.linalg.inv(sigma_0)
    sigma1_inv = np.linalg.inv(sigma_1)
    
    Q = sigma0_inv - sigma1_inv
    Q=Q/2
    
    
    beta = sigma1_inv @ mu_1 - sigma0_inv @ mu_0 
    alpha = 1/2 * (mu_0.T @ sigma0_inv @  mu_0 - mu_1.T @ sigma1_inv @  mu_1) 
     + np.log(pi/(1-pi)) + 1/2 * (np.log(np.linalg.det(sigma_0)) -
                                  np.log(np.linalg.det(sigma_1)))
    
    print("Q", round(Q,2), "\nbeta:", round(beta,2), "\nalpha:", round(alpha,2))
    return Q, beta, alpha


def f(x,Q,beta,alpha):
        return Q[0,0]*x[0]**2 + x[0]*x[1]*(Q[1,0] + Q[0,1]) + Q[1,1]*x[1]**2 +
         beta[0]* x[0] + beta[1]* x[1] + alpha

def sep_line(train,test, Q, beta, alpha):

    x_train = train[:,:-1]
    y_train = train[:,2]
    
    y = y_train.reshape(-1,1)
    x_test = test[:,:-1]
    y_test = test[:,2]

    x_min, x_max = np.min(x_train[:,0]), np.max(x_train[:,0])
    y_min, y_max = np.min(x_train[:,1]), np.max(x_train[:,1])
    
     
    boundary_x0 = np.linspace(x_min, x_max, 1000)
    boundary_x1 = np.linspace(y_min, y_max, 1000)
    XX = np.meshgrid(boundary_x0, boundary_x1)
    Z = f(XX,Q,beta,alpha)

    
    Xt_1 = x_test[y_test == 1]
    Xt_0 = x_test[y_test == 0]
    
    x_min, x_max = np.min(x_test[:,0]), np.max(x_test[:,0])
    y_min, y_max = np.min(x_test[:,1]), np.max(x_test[:,1])
         
    boundary_xt0 = np.linspace(x_min, x_max, 1000)
    boundary_xt1 = np.linspace(y_min, y_max, 1000)
    XXt = np.meshgrid(boundary_xt0,boundary_xt1)
    Zt = f(XXt,Q, beta, alpha)

    return boundary_x0, boundary_x1, boundary_xt0, boundary_xt1, Z, Zt


def pred_QDA(train, test, Q, beta, alpha):
    x_train = train[:,:-1]
    y_train = train[:,2]
    
    y = y_train.reshape(-1,1)
    x_test = test[:,:-1]
    y_test = test[:,2]
    pred = np.array([f(i,Q,beta,alpha) for i in x_train])
    pred = np.where(np.array(pred)>0,0,1)
    misclassification = 1- sum(pred != y_train)/len(y_train)
    
    pred = np.array([f(i,Q, beta, alpha) for i in x_test])
    pred = np.where(np.array(pred)>0,0,1)
    misclass = 1- sum(pred != y_test)/len(y_test)

    print("Misclassification on train set: ", misclassification)
    print("\n")
    print("Misclassification on test set: ", misclass)
    print("\n")

Let's estimate parameters for each training set using the train function, and then let's determine the line defined by P(Y=1|x) = 0.5 for each training and testing sets. Then let's plot the datasets with the separating line. 

In [0]:
model = "QDA"
train, test = trainA, testA
Q, beta, alpha = train_QDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt, Z, Zt = sep_line(train,test, Q, beta, alpha)
plot_classification(train,test,boundary_x,boundary_y,"A", linear = False,
                        model,Z=Z,Zt=Zt)
pred_linreg(train,test, Q, beta, alpha)

In [0]:
model = "QDA"
train, test = trainA, testA
Q, beta, alpha = train_QDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt, Z, Zt = sep_line(train,test, Q, beta, alpha)
plot_classification(train,test,boundary_x,boundary_y,"B", linear = False,
                        model,Z=Z,Zt=Zt)
pred_linreg(train,test, Q, beta, alpha)

In [0]:
model = "QDA"
train, test = trainA, testA
Q, beta, alpha = train_QDA(train,test)
boundary_x, boundary_y, boundary_xt, boundary_yt, Z, Zt = sep_line(train,test, Q, beta, alpha)
plot_classification(train,test,boundary_x,boundary_y,"C", linear = False,
                        model,Z=Z,Zt=Zt)
pred_linreg(train,test, Q, beta, alpha)